# Coursera Week 5 Assignment


### Import the libraries used for the asignment

In [1]:
#Import libraries
!pip install beautifulsoup4
!pip install bs4
from urllib.request import urlopen

from bs4 import BeautifulSoup
import datetime
import re
import numpy as np
import pandas as pd
import csv

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::altair==4.0.1=py_0
  - defaults/noarch::autovizwidget==0.12.7=py_0
  - defaults/win-64::bokeh==1.1.0=py36_0
  - defaults/win-64::bottleneck==1.2.1=py36h452e1ab_1
  - defaults/win-64::cupy==4.1.0=py36h137e5fc_0
  - defaults/win-64::datashape==0.5.4=py36_1
  - defaults/win-64::descartes==1.1.0=py36_0
  - conda-forge/noarch::folium==0.5.0=py_0
  - defaults/win-64::gdal==2.3.3=py36hdf43c64_0
  - defaults/win-64::h5py==2.9.0=py36h5e291fa_0
  - conda-forge/noarch::hdijupyterutils==0.12.9=py_0
  - conda-forge/win-64::mkl_fft==1.0.13=py36hfa6e2cd_1
  - conda-forge/win-64::pandas==1.0.1=py36he350917_0
  - https://repo.anaconda.com/pkgs/main/noarch/noarch::snuggs==1.4.3=py_0
  - conda-forge/noarch::vincent==0.4.4=py_1

  - https://repo.anaconda.com/pkgs/main/noarch/noarch::sphinxcontrib-applehelp-1.0.1-py_0, ht

ERROR conda.core.link:_execute(700): An error occurred while installing package 'conda-forge::matplotlib-base-3.1.1-py36h2852a4a_1'.

[Errno 13] Permission denied: 'C:\\Users\\sB006047682\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\matplotlib\\_contour.cp36-win_amd64.pyd'
()



ImportError: cannot import name 'KMeans'

## 1. Get Toronto Postal Code information from Wikipedia

### 1.1 HTML Formatting Functions 

In [ ]:
def remove_tags(data_arr_list):
    tags = ["<td>", "</td>", "\n", "td>" , "</td", "]]"]
    for i in range(0, len(data_arr_list)):
        for j in range(0, len(tags)):
            if str(tags[j]) in str(data_arr_list[i]):
                data_arr_list[i] = data_arr_list[i].replace(tags[j], "")
                if 'title="' in str(data_arr_list[i]):
                    data_arr_list[i] = str(data_arr_list[i]).split('title="')[1].split('">')[0]
    
    return (data_arr_list)

In [ ]:
def compile_postal(data_arr_list):

    #Compare the postal code to the next one in order
    for i in range (0, len(data_arr_list)-3, 3):

        if str(data_arr_list[i]) == str(data_arr_list[i+3]):
            #Add to the current postal code
            if str(data_arr_list[i+4]) not in data_arr_list[i+1]:
                data_arr_list[i+1] = str(data_arr_list[i+1]) + ", " + str(data_arr_list[i+4])
            if str(data_arr_list[i+5]) not in data_arr_list[i+2]:
                data_arr_list[i+2] = str(data_arr_list[i+2]) + ", " + str(data_arr_list[i+5])
            
            #Remove old entry(s)
            del(data_arr_list[i+3])
            del(data_arr_list[i+3])
            del(data_arr_list[i+3])
            
            data_arr_list = compile_postal(data_arr_list)
            
            break
            
    return data_arr_list

In [ ]:
# Description: Drop borough rows that are N/A

def drop_na_borough(data_arr_list):

    for i in range (1, len(data_arr_list)-1, 3):
        if str(data_arr_list[i]) == 'Not assigned':
            
            #Remove the row
            del(data_arr_list[i-1])
            del(data_arr_list[i-1])
            del(data_arr_list[i-1])
            
            data_arr_list = drop_na_borough(data_arr_list)
            break
            
    return data_arr_list

In [ ]:
def neighborhood_borough(data_arr_list):
    
    for i in range (2, len(data_arr_list), 3):
        if str(data_arr_list[i]) == 'Not assigned':
            
            data_arr_list[i] = str(data_arr_list[i-1])
            data_arr_list = neighborhood_borough(data_arr_list)
            
            break
            
    return data_arr_list

### 1.2 Identify Postal Code Information from the Wikipedia page

In [ ]:
# specify the url
quote_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# query the website and return the html to the variable ‘page’
page = urlopen(quote_page)

# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(page, "html.parser")

#Define array to hold all of the data points
data_arr = []

#Get the first table in the html
data = soup.findAll('table')

#assign the cells to the array
for row in data:
    for item in row.findAll('td'):
        if "<td>" in str(item):
            data_arr.append(str(item))

#Remove the last element in the list as it is invalid
data_arr.pop()            

#Clean up the tags and data points

#Remove HTML tags
data_arr = remove_tags(data_arr)

#Compile postal codes
data_arr = compile_postal(data_arr)

#Drop Not assigned boroughs
data_arr = drop_na_borough(data_arr)

#Assign borough to n/a neighborhoods
data_arr = neighborhood_borough(data_arr)

## 2. Create a Pandas Dataframe with Toronto data

### 2.1 Display the Dataframe with Wikipedia Information

In [ ]:
#Create a dictionary
toronto_dict = {'Postal_Code':data_arr[0::3], 'Borough': data_arr[1::3], 
                                     'Neighborhood':data_arr[2::3] }

#Pandas Data frame
toronto_df = pd.DataFrame.from_dict(toronto_dict)

#*********Uncomment these lines to focus only on those boroughs in Toronto - containing the word Toronto*********#
#toronto_df = toronto_df[toronto_df['Borough'].str.contains("Toronto")==True]
#toronto_df.reset_index(drop=True, inplace=True)

#Print the shape of the new frame and display the first 5 rows
print(toronto_df.shape)

toronto_df.head()

### 2.2 Add location information to the Dataframe

In [ ]:
#Add the Latitude and Longitude columns to the table and initialize with placeholder information
toronto_df['Latitude'] = 'Not Set'
toronto_df['Longitude'] = 'Not Set'

#Open file containing the geospacial coordinates for Toronto
with open('Geospatial_Coordinates.csv', 'r') as csvfile:
    geo_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in geo_reader:
        #Find the postal code in the fame and add coordinates
        toronto_df.loc[toronto_df['Postal_Code'] == str(row[0]), "Latitude"] = str(row[1])
        toronto_df.loc[toronto_df['Postal_Code'] == str(row[0]), "Longitude"] = str(row[2])
        
#Set type to numeric
toronto_df['Latitude'] = pd.to_numeric(toronto_df['Latitude'])
toronto_df['Longitude'] = pd.to_numeric(toronto_df['Longitude'])


toronto_df.head()

### 2.3 Add population information to the Dataframe

In [ ]:
#Add the population columns to the table and initialize with placeholder information
toronto_df['Population'] = 'Not Set'

#Open file containing the geospacial coordinates for Toronto
with open('Canada Population.csv', 'r') as csvfile:
    geo_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in geo_reader:
        #Find the postal code in the fame and add coordinates
        toronto_df.loc[toronto_df['Postal_Code'] == str(row[0]), "Population"] = str(row[4])

#For those without population data, remove them, from the dataframe
toronto_df = toronto_df[toronto_df['Population'].str.contains("Not Set")==False]
toronto_df.reset_index(drop=True, inplace=True)
toronto_df['Population'] = pd.to_numeric(toronto_df['Population'])

toronto_df.head()

### 2.4 Remove underpopulated postal codes from the dataframe

In [ ]:
#Remove rows with < 25000 population
clip = 20000

toronto_df = toronto_df[toronto_df['Population'].ge(25000)]
toronto_df ['Population'] = pd.to_numeric(toronto_df ['Population'])
toronto_df.reset_index(drop=True, inplace=True)

toronto_df.head()

# 3. Get nearby venues from Foursquare

### 3.1 Foursquare credentials

In [ ]:
CLIENT_ID = 'B0K442ZABARQK0HVKEPBPEPLSDNBXTGUQ2PEOEU0AMDN4YAJ' # your Foursquare ID
CLIENT_SECRET = 'TOTW50KQOA02KK343155NVTYIGO4XW3PP2SDNYZEBZYV4Z3D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### 3.2 Foursquare function to pull nearby values

In [ ]:
## Cognitive Class.ai
## Segmenting and Clustering Neighborhoods in New York City
## Note: This function is taken from : https://labs.cognitiveclass.ai/tools/jupyterlab/lab/tree/labs/DP0701EN/DP0701EN-3-3-2-Neighborhoods-New-York-py-v1.0.ipynb
## I do not take credit for writing the below function
## I have used this function and made changes where nessesary for use in this project

#This function will get the near-by venues of a location using coordinates

def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    remove = []
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #Log postal codees without nearby addresses 
        if not results:
            remove.append(name)

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal_Code', 
                  'Postal_Latitude', 
                  'Postal_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues, remove)

### 3.3 Get venues and add to the Dataframe

In [ ]:
#Radius of 750m and a limit of 100 venues
radius = 750
LIMIT = 100


#Get the venues near Toronto postal codes
toronto_venues,remove = getNearbyVenues(names=toronto_df['Postal_Code'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude'],
                                   radius = radius
                                  )

#Remove those postal codes with no nearby venues
for item in remove:
    indexNames = toronto_df[ toronto_df['Postal_Code'] == item ].index
    # Delete these row indexes from dataFrame
    toronto_df.drop(indexNames , inplace=True)

### 3.4 Examine the dataframe

In [ ]:
# Display the shape of the dataframe and the first 5 rows
toronto_venues.head(5)

In [ ]:
#Determine how many unique categories there are 
print('There are {} uniques categories.'.format(len(toronto_venues['Venue_Category'].unique())))

In [ ]:
#Display the categories:
for i in range (0, len(toronto_venues['Venue_Category'].unique())):
    print(toronto_venues['Venue_Category'].unique()[i])

# 4. Assign points to the venues

### 4.1 Define function to determine point value for venues

In [ ]:
def get_points(venue):
    venue = venue.lower() 
    venue = venue.split()
    value = 1
    
    two_points = ["bar", "pub", "bus", "transport", "rail", "plane" \
                        , "boat", "airport", "ferry", "station"]
    three_points = ["church", "club", "hotel", "hostel", "community", \
                        "museaum", "court", "field", "stadium", "sport", \
                        "playground", "theatre", "gallery", "center", "hall", \
                        "event", "park"]
    four_points = ["store", "shop", "mall", "plaza", "market", "grocery"]
    five_points = ["office", "school"]
    minus_two = ["restaurant", "joint", "diner", "food"]
    minus_ten = ["adult"]
    
    for word in venue:
        if word in two_points:
            if 2 > value:
                value = 2

        if word in three_points:
            if 3 > value:
                value = 3

        if word in four_points:
            if 4 > value:
                value = 4

        if word in five_points:
            if 5 > value:
                value = 5

        if word in minus_two:
            if value > -2:
                value = -2  

        if word in minus_ten:
            if value > -10:    
                value = -10
    
    return (value)

### 4.2 Add column to dataframe to record points

In [ ]:
#Add the points column to the df
toronto_venues['Point_Value'] = "Not Set"

#Get the point value for each venue
for i in range(0, len(toronto_venues)):
    toronto_venues["Point_Value"][i] = get_points(str(toronto_venues["Venue_Category"][i]))
    

toronto_venues['Point_Value'] = pd.to_numeric(toronto_venues['Point_Value'])

toronto_venues.head()

### 4.3 Add sum of venue points to the postal code in toronto_df

In [ ]:
#Add the points column to the toronto_df
toronto_df['Point_Value'] = "Not Set"

#Sum the venues points for each postal code and add to toronto_df
for i in range(0, len(toronto_df)):
    toronto_df["Point_Value"][i] = toronto_venues[ toronto_venues['Postal_Code'] \
                                           == toronto_df["Postal_Code"][i]]["Point_Value"].sum() 

toronto_df['Point_Value'] = pd.to_numeric(toronto_df['Point_Value'])

toronto_df.head()

### 4.4 Sort the df and analyze the results

In [ ]:
#Sort the df in descending order
toronto_df = toronto_df.sort_values(by='Point_Value', ascending=False)

#display the df
toronto_df.head(5)

In [ ]:
# use the function regplot to make a scatterplot
sns.regplot(x=toronto_df["Population"], y=toronto_df["Point_Value"], fit_reg=False)

### 4.5 Normalize the population and scale the points

In [ ]:
#Normalize the data
xmax = toronto_df['Population'].max().astype(int)
xmin = toronto_df['Population'].min().astype(int)


for i in range(0, len(toronto_df)):
    toronto_df["Point_Value"][i] = (1-(toronto_df['Population'][i].astype(float) - xmin)/(xmax-xmin))*toronto_df["Point_Value"][i]

#Sort the df in descending order
toronto_df = toronto_df.sort_values(by='Point_Value', ascending=False)

#Reset index
toronto_df.reset_index(drop=True, inplace=True)

toronto_df.head()

In [ ]:
#use the function regplot to make a scatterplot
sns.regplot(x=toronto_df["Population"], y=toronto_df["Point_Value"], fit_reg=False)

# 5. Determine and map best locations

### 5.1 Show top three locations and attributes

In [ ]:
for i in range (0, 3):
    print("[" + str(i+1) + "]")
    print("Postal Code: " + str(toronto_df['Postal_Code'][i]))
    print("Borough: " + str(toronto_df['Borough'][i]))
    print("Neighborhood: " + str(toronto_df['Neighborhood'][i]))
    print("Latitude: " + str(toronto_df['Latitude'][i]) + ", Longitude: " + str(toronto_df['Longitude'][i]))
    print("Population: " + str(toronto_df['Population'][i]))
    print("\n\n")
    

In [ ]:
latitude = 43.7
longitude = -79.3832

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood'], [1,2,3]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color='orange',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters